In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from pathlib import Path
from pyprojroot import here


In [ ]:
import pyprojroot 

pyprojroot.__file__

In [ ]:
import jax

In [ ]:
from jax_unirep import get_reps

In [ ]:

from patch_gnn.data import load_ghesquire

data = load_ghesquire()

In [ ]:
from functools import partial, singledispatch


@singledispatch
def split_delimiter(x, delimiter=";"):
    """Split delimiter helper function."""
    raise NotImplementedError("Unsupported type!")


@split_delimiter.register(float)
def _split_delimiter(x, delimiter=";"):
    """Split delimiter helper function for floats."""
    return x


@split_delimiter.register(str)
def _split_delimiter(x, delimiter=";"):
    """Split delimiter helper function for strings."""
    return x.split(delimiter)

In [ ]:
import janitor
processed_data = (
    data.dropna(subset=["accession"])
    .transform_column("isoforms", split_delimiter)
    .explode("isoforms")
    .transform_column("isoforms", partial(split_delimiter, delimiter=" ("))
    .transform_column("isoforms", lambda x: x[0] if isinstance(x, list) else x)
    .transform_column(
        "isoforms", lambda x: x.strip(" ") if isinstance(x, str) else x
    )
    .drop_duplicates("sequence")
)


In [ ]:
processed_data

In [ ]:
from jax_unirep import get_reps
import jax.numpy as np

In [ ]:
seqs = processed_data["sequence"].tolist()

In [ ]:
reps = get_reps(seqs)

In [ ]:
Xs = reps[0]
ys = processed_data["ox_fwd_logit"]

In [ ]:
Xs.shape, ys.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xs, ys, random_state=444, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rfr = RandomForestRegressor(n_jobs=-1)
rfr.fit(X_train, y_train)

In [ ]:
preds = rfr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error as mse, explained_variance_score as evs

In [ ]:
evs(y_test, preds)

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(y_test, preds)
plt.plot([-4, 4], [-4, 4])
plt.title("UniRep + RF")
plt.xlabel("Ground truth")
plt.ylabel("Predicted")